In [8]:
import numpy as np

#load eigenvector information
#for the purpose of easy calculation, here we load structure information

from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

In [9]:
with open('gulp.eig','r') as eig:
    lines=eig.readlines()
    #header: number of atom, atom position, number of k-point, number of modes
    ref_natom=int(lines[0])
    if natom!=ref_natom:
        assert "number of atom is different"
    ref_nkpoints=int(lines[natom+1])
    if ref_nkpoints!=0:
        assert "number of kpoint is not zero"
    ref_nmodes=int(lines[natom+2])
    if ref_nmodes!=3*natom:
        assert "number of modes seems strange... format error?"
    
    #lines[natom+3] --> Kpoint coord

    start=natom+4
    mode_block=natom+10
    nmodes=3*natom
    omega=np.zeros(nmodes)
    eigenvector=np.zeros((nmodes,natom,3))

    for imode in range(nmodes):
        omega[imode]=float(lines[start+1])
        eigenv_mode=np.loadtxt(lines[start+2: start+2+natom])
        eigenvector[imode]=eigenv_mode

        start=start+2+natom 

In [10]:
omega

array([-1.40000000e-05, -6.00000000e-06,  1.40000000e-05,  2.21940168e+02,
        2.21940168e+02,  2.21940168e+02,  2.21940168e+02,  2.21940168e+02,
        2.21940168e+02,  4.29619339e+02,  4.29619339e+02,  4.29619339e+02,
        4.29619339e+02,  4.29619339e+02,  4.29619339e+02,  5.16419661e+02,
        5.16419661e+02,  5.16419661e+02,  5.16419661e+02,  5.16419661e+02,
        5.16419661e+02,  5.89804580e+02,  5.89804580e+02,  5.89804580e+02])

In [11]:
eigenvector[0]

array([[-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464],
       [-0.0810933 ,  0.34412729,  0.00053464]])

In [12]:
eigen_flat=np.reshape(eigenvector,(nmodes, natom*3))

In [13]:
eigen_flat[0]

array([-0.0810933 ,  0.34412729,  0.00053464, -0.0810933 ,  0.34412729,
        0.00053464, -0.0810933 ,  0.34412729,  0.00053464, -0.0810933 ,
        0.34412729,  0.00053464, -0.0810933 ,  0.34412729,  0.00053464,
       -0.0810933 ,  0.34412729,  0.00053464, -0.0810933 ,  0.34412729,
        0.00053464, -0.0810933 ,  0.34412729,  0.00053464])

In [14]:
eigen_flat[1]

array([0.00939437, 0.0016647 , 0.35342464, 0.00939437, 0.0016647 ,
       0.35342464, 0.00939437, 0.0016647 , 0.35342464, 0.00939437,
       0.0016647 , 0.35342464, 0.00939437, 0.0016647 , 0.35342464,
       0.00939437, 0.0016647 , 0.35342464, 0.00939437, 0.0016647 ,
       0.35342464, 0.00939437, 0.0016647 , 0.35342464])

In [15]:
eigen_flat[3]

array([ 0.06927493,  0.17493567, -0.21775762, -0.06927493, -0.26847225,
       -0.30628705,  0.03593061, -0.17493567,  0.30628705, -0.03593061,
        0.26847225,  0.21775762,  0.26596868, -0.24535017,  0.00583448,
       -0.26596868, -0.27869449, -0.09937106, -0.17743925,  0.24535017,
        0.09937106,  0.17743925,  0.27869449, -0.00583448])

In [16]:
masses=atoms.get_masses()
dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin

In [19]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock

In [20]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]

In [21]:
Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

for i in range(natom):
    for j in range(natom):
        Vijx=Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx

        Vijy=Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy

        Vijz=Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz

In [23]:
flatVx=np.reshape(Vx.transpose(0,2,1,3),(24,24))
flatVy=np.reshape(Vy.transpose(0,2,1,3),(24,24))
flatVz=np.reshape(Vz.transpose(0,2,1,3),(24,24))

#check
GULP_Vijz=np.loadtxt('Vijz.txt').reshape((24,24))
np.sum(flatVz-GULP_Vijz)

-2.7755575615628914e-17

In [25]:
eigen_Vx=np.dot(eigen_flat.T,flatVx)
eigen_Vx.shape

(24, 24)

In [26]:
eigen_Vx_eigen=np.dot(eigen_Vx,eigen_flat)

It seems the result not matches with that of GULP...

In [28]:
np.max(eigen_Vx_eigen)

0.3704920325297108